In [1]:
from typing import Iterable
from functools import reduce

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option("display.max_columns", None)

In [2]:
def get_matches(
    matches_data,
    league: str | Iterable[str] = None,
    split: str | Iterable[str] = None,
    playoffs=None,
    patch: str | Iterable[str] = None,
):
    df = matches_data
    if league is not None:
        if not isinstance(league, str) and isinstance(league, Iterable):
            df = df.loc[df["league"].isin(league)]
        else:
            df = df.loc[df["league"] == league]
    if split is not None:
        if not isinstance(split, str) and isinstance(split, Iterable):
            df = df.loc[df["split"].isin(split)]
        else:
            df = df.loc[df["split"] == split]
    if playoffs is not None:
        df = df.loc[df["playoffs"] == playoffs]
    if patch is not None:
        if not isinstance(patch, str) and isinstance(patch, Iterable):
            df = df.loc[df["patch"].isin(patch)]
        else:
            df = df.loc[df["patch"] == patch]

    return df


def get_rc(i, cols):
    r = i // cols + 1
    c = i % cols + 1

    return (r, c)


def get_rows_columns(df, c=5):
    COLUMNS = c
    d, m = divmod(df.shape[0], COLUMNS)
    # ROWS = d + (1 if m > 0 else m)
    ROWS = df.shape[0]

    return ROWS, COLUMNS


def get_fig_subplots(df, ROWS, COLUMNS):
    titles = reduce(
        lambda x, y: x + y, [[value] * COLUMNS for value in df.index.values]
    )

    fig = make_subplots(
        rows=ROWS,
        cols=COLUMNS,
        subplot_titles=titles,
        specs=[[{"type": "polar"} for _ in range(COLUMNS)] for _ in range(ROWS)],
    )

    return fig


def get_min_max_value(df, columns):
    min_value = df[columns].unstack().min()
    max_value = df[columns].unstack().max()

    return min_value, max_value


def get_theta_list(df, name, origin_columns):
    return [
        f"{col} - {round(value, ndigits=2)}"
        for col, value in zip(origin_columns, df.loc[name, origin_columns].values)
    ]

In [3]:
matches_data = pd.read_csv(
    "./csv/oracleselixir_match_data/2023_LoL_esports_match_data_from_OraclesElixir.csv",
    dtype={"patch": "object"},
)
matches_data["kda"] = (
    matches_data[["kills", "assists"]].sum(axis=1).divide(matches_data["deaths"])
)
matches_data["kd"] = matches_data["kills"].divide(matches_data["deaths"])
matches_data["gpm"] = matches_data["totalgold"].divide(matches_data["gamelength"] / 60)
matches_data.reset_index(drop=False, inplace=True)
matches_data.shape

(74832, 127)

In [4]:
columns = [
    "index",
    "gameid",
    "teamid",
    "participantid",
    "position",
    "gamelength",
    "totalgold",
]
merged = pd.merge(
    matches_data[columns],
    matches_data[columns],
    on=["gameid", "position"],
    how="inner",
)
merged = merged.loc[merged["participantid_x"] != merged["participantid_y"]]
merged.head(20)

,index_x,gameid,teamid_x,participantid_x,position,gamelength_x,totalgold_x,index_y,teamid_y,participantid_y,gamelength_y,totalgold_y
1,0,ESPORTSTMNT06_2753012,oe:team:0ade5e44c23039bca133eee58ec1b83,1,top,2612,18855,5,oe:team:14ad76b8d9e647d4b29c3d26ecd29c9,6,2612,11796
2,5,ESPORTSTMNT06_2753012,oe:team:14ad76b8d9e647d4b29c3d26ecd29c9,6,top,2612,11796,0,oe:team:0ade5e44c23039bca133eee58ec1b83,1,2612,18855
5,1,ESPORTSTMNT06_2753012,oe:team:0ade5e44c23039bca133eee58ec1b83,2,jng,2612,12082,6,oe:team:14ad76b8d9e647d4b29c3d26ecd29c9,7,2612,11711
6,6,ESPORTSTMNT06_2753012,oe:team:14ad76b8d9e647d4b29c3d26ecd29c9,7,jng,2612,11711,1,oe:team:0ade5e44c23039bca133eee58ec1b83,2,2612,12082
9,2,ESPORTSTMNT06_2753012,oe:team:0ade5e44c23039bca133eee58ec1b83,3,mid,2612,15722,7,oe:team:14ad76b8d9e647d4b29c3d26ecd29c9,8,2612,14129
10,7,ESPORTSTMNT06_2753012,oe:team:14ad76b8d9e647d4b29c3d26ecd29c9,8,mid,2612,14129,2,oe:team:0ade5e44c23039bca133eee58ec1b83,3,2612,15722
13,3,ESPORTSTMNT06_2753012,oe:team:0ade5e44c23039bca133eee58ec1b83,4,bot,2612,17332,8,oe:team:14ad76b8d9e647d4b29c3d26ecd29c9,9,2612,16823
14,8,ESPORTSTMNT06_2753012,oe:team:14ad76b8d9e647d4b29c3d26ecd29c9,9,bot,2612,16823,3,oe:team:0ade5e44c23039bca133eee58ec1b83,4,2612,17332
17,4,ESPORTSTMNT06_2753012,oe:team:0ade5e44c23039bca133eee58ec1b83,5,sup,2612,8816,9,oe:team:14ad76b8d9e647d4b29c3d26ecd29c9,10,2612,8286
18,9,ESPORTSTMNT06_2753012,oe:team:14ad76b8d9e647d4b29c3d26ecd29c9,10,sup,2612,8286,4,oe:team:0ade5e44c23039bca133eee58ec1b83,5,2612,8816


In [5]:
matches_data["gdpm"] = 0
for row in merged.itertuples():
    gdpm = (row.totalgold_x - row.totalgold_y) / (row.gamelength_x / 60)
    matches_data.loc[row.index_x, "gdpm"] = gdpm
    matches_data.loc[row.index_y, "gdpm"] = -gdpm
matches_data["gdpm"]

0        162.151608
1          8.522205
2         36.592649
3         11.692190
4         12.174579
            ...    
74827    135.205748
74828    -60.548661
74829     63.723057
74830   -396.721097
74831    396.721097
Name: gdpm, Length: 74832, dtype: float64

In [6]:
matches_data.columns

Index(['index', 'gameid', 'datacompleteness', 'url', 'league', 'year', 'split',
       'playoffs', 'date', 'game',
       ...
       'killsat15', 'assistsat15', 'deathsat15', 'opp_killsat15',
       'opp_assistsat15', 'opp_deathsat15', 'kda', 'kd', 'gpm', 'gdpm'],
      dtype='object', length=128)

In [7]:
matches_data.head()

,index,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,participantid,side,position,playername,playerid,teamname,teamid,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,chemtechs,hextechs,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,kda,kd,gpm,gdpm
0,0,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,1,Blue,top,Wylenz,oe:player:60aff1184bec1d2b2efdae84f5b6e3e,Klanik Esport,oe:team:0ade5e44c23039bca133eee58ec1b83,Jax,Sylas,Caitlyn,Wukong,Akali,Yone,2612,1,4,0,6,13,7,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2986,0.4594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,14283,328.0934,0.150027,719.5406,878.9127,20,0.4594,5,0.1149,11,49,1.1256,18855,13251,304.3874,0.295868,15955,NaN,399.0,367.0,32,NaN,NaN,9.1654,3163.0,4772.0,83.0,3087.0,4814.0,81.0,76.0,-42.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5059.0,7946.0,143.0,4737.0,7683.0,131.0,322.0,263.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,inf,inf,433.116386,162.151608
1,1,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,2,Blue,jng,Julbu,oe:player:fd78e127e45463dcfc2ea3836af0335,Klanik Esport,oe:team:0ade5e44c23039bca133eee58ec1b83,Poppy,Sylas,Caitlyn,Wukong,Akali,Yone,2612,1,2,2,4,13,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2986,0.4594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,6219,142.8560,0.065324,847.3047,1513.9663,18,0.4135,11,0.2527,17,61,1.4012,12082,6478,148.8055,0.144640,11675,NaN,159.0,23.0,136,NaN,NaN,3.6524,3035.0,3302.0,54.0,2948.0,3140.0,66.0,87.0,162.0,-12.0,0.0,0.0,0.0,0.0,0.0,0.0,4325.0,4637.0,74.0,4682.0,5960.0,117.0,-357.0,-1323.0,-43.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,277.534456,8.522205
2,2,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,3,Blue,mid,Sintax,oe:player:baf7147fedeec5de54ca1f240952a3f,Klanik Esport,oe:team:0ade5e44c23039bca133eee58ec1b83,Taliyah,Sylas,Caitlyn,Wukong,Akali,Yone,2612,1,2,2,11,13,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2986,0.4594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,27028,620.8576,0.283899,365.3522,323.7289,23,0.5283,10,0.2297,7,49,1.1256,15722,10118,232.4196,0.225914,14525,NaN,337.0,287.0,50,NaN,NaN,7.7412,3117.0,4927.0,85.0,3455.0,5174.0,103.0,-338.0,-247.0,-18.0,0.0,0.0,0.0,0.0,0.0,0.0,4956.0,7772.0,136.0,5435.0,8096.0,162.0,-479.0,-324.0,-26.0,0.0,0.0,0.0,0.0,0.0,0.0,6.5,1.0,361.148545,36.592649
3,3,ESPORTSTMNT06_2753012,complete,NaN,LFL2,2023,Spring,0,2023-01-10 17:07:16,1,13.01,4,Blue,bot,Axelent,oe:player:8204ca38dc1c42012b5d53131271eb1,Klanik Esport,oe:team:0ade5e44c23039bca133eee58ec1b83,Ezreal,Sylas,Caitlyn,Wukong,Akali,Yone,2612,1,5,1,7,13,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2986,0.4594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,Na

In [8]:
matches = get_matches(matches_data, "LCK", "Summer")
matches.head()

,index,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,participantid,side,position,playername,playerid,teamname,teamid,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,chemtechs,hextechs,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,kda,kd,gpm,gdpm
67572,67572,ESPORTSTMNT03_3167508,complete,NaN,LCK,2023,Summer,0,2023-06-07 08:11:03,1,13.10,1,Blue,top,Kiin,oe:player:03daae1844b9ad257324c3eedf41e28,KT Rolster,oe:team:d723992568441c6bafa2658129eade7,Malphite,Vi,Maokai,Lucian,Lulu,Ornn,1894,1,2,1,9,12,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3801,0.5702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,17628,558.4372,0.277716,661.4572,1947.4974,11,0.3485,10,0.3168,6,37,1.1721,12665,8526,270.0950,0.221542,11800,NaN,285.0,281.0,4,NaN,NaN,9.0285,3373.0,5212.0,95.0,3676.0,5212.0,98.0,-303.0,0.0,-3.0,0.0,0.0,0.0,0.0,0.0,0.0,5153.0,8046.0,146.0,5274.0,7985.0,147.0,-121.0,61.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,2.0,401.214361,53.315734
67573,67573,ESPORTSTMNT03_3167508,complete,NaN,LCK,2023,Summer,0,2023-06-07 08:11:03,1,13.10,2,Blue,jng,Cuzz,oe:player:f9fe6d28d5b67bf9ac673b2cc8edf15,KT Rolster,oe:team:d723992568441c6bafa2658129eade7,Viego,Vi,Maokai,Lucian,Lulu,Ornn,1894,1,5,1,5,12,6,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.3801,0.5702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,13080,414.3611,0.206065,806.1035,970.7075,14,0.4435,20,0.6336,22,63,1.9958,12416,8277,262.2070,0.215072,11225,NaN,214.0,23.0,191,NaN,NaN,6.7793,3334.0,3779.0,78.0,3256.0,3652.0,71.0,78.0,127.0,7.0,0.0,0.0,1.0,0.0,0.0,0.0,4884.0,5891.0,117.0,5011.0,5448.0,106.0,-127.0,443.0,11.0,0.0,0.0,1.0,0.0,0.0,0.0,10.0,5.0,393.326294,61.805702
67574,67574,ESPORTSTMNT03_3167508,complete,NaN,LCK,2023,Summer,0,2023-06-07 08:11:03,1,13.10,3,Blue,mid,Bdd,oe:player:30361653a485a9609fb2d3aa073282c,KT Rolster,oe:team:d723992568441c6bafa2658129eade7,Annie,Vi,Maokai,Lucian,Lulu,Ornn,1894,1,2,2,10,12,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3801,0.5702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,15295,484.5301,0.240961,330.3168,291.0982,24,0.7603,7,0.2218,13,42,1.3305,13085,8946,283.4002,0.232456,12475,NaN,293.0,293.0,0,NaN,NaN,9.2819,3526.0,5066.0,99.0,3509.0,4208.0,80.0,17.0,858.0,19.0,0.0,0.0,0.0,1.0,0.0,0.0,5425.0,8471.0,163.0,5265.0,7040.0,134.0,160.0,1431.0,29.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,1.0,414.519535,47.771911
67575,67575,ESPORTSTMNT03_3167508,complete,NaN,LCK,2023,Summer,0,2023-06-07 08:11:03,1,13.10,4,Blue,bot,Aiming,oe:player:85104684d1eb7f4c0ce393441a0b669,KT Rolster,oe:team:d723992568441c6bafa2658129eade7,Ezreal,Vi,Maokai,Lucian,Lulu,Ornn,1894,1,2,1,3,12,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3801,0.5702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,N

In [9]:
teams_data = matches.loc[matches["position"] == "team"]
df = pd.read_csv("./csv/scoreboard_games/2023_scoreboard_games.csv")
df = df.loc[df["OverviewPage"] == "LCK/2023 Season/Summer Season"]
print(teams_data.shape, df.shape, teams_data.shape[0] == df.shape[0] * 2)
teams_data.head()

(94, 128) (47, 37) True


,index,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,participantid,side,position,playername,playerid,teamname,teamid,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,chemtechs,hextechs,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,killsat10,assistsat10,deathsat10,opp_killsat10,opp_assistsat10,opp_deathsat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15,kda,kd,gpm,gdpm
67582,67582,ESPORTSTMNT03_3167508,complete,NaN,LCK,2023,Summer,0,2023-06-07 08:11:03,1,13.10,100,Blue,team,NaN,NaN,KT Rolster,oe:team:d723992568441c6bafa2658129eade7,NaN,Vi,Maokai,Lucian,Lulu,Ornn,1894,1,12,6,35,12,6,2.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,0.3801,0.5702,0.0,3.0,1.0,3.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,9.0,3.0,0.0,0.0,3.0,4.0,2.0,0.0,63475,2010.8237,NaN,2284.3400,3502.7138,101,3.1996,54,1.7107,55,247,7.8247,59180,38483,1219.1024,NaN,54633,0.068589,NaN,906.0,207,NaN,NaN,35.2587,15745.0,19908.0,359.0,16648.0,19255.0,365.0,-903.0,653.0,-6.0,0.0,0.0,1.0,1.0,1.0,0.0,23707.0,31810.0,574.0,24376.0,29938.0,563.0,-669.0,1872.0,11.0,0.0,0.0,1.0,1.0,1.0,0.0,7.833333,2.000000,1874.762408,212.122492
67583,67583,ESPORTSTMNT03_3167508,complete,NaN,LCK,2023,Summer,0,2023-06-07 08:11:03,1,13.10,200,Red,team,NaN,NaN,Hanwha Life Esports,oe:team:3a1d18f46bcb3716ebcfcf4ef068934,NaN,Milio,Zeri,Neeko,Kha'Zix,Jax,1894,0,6,12,15,6,12,1.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,0.1901,0.5702,1.0,1.0,3.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,1.0,2.0,0.0,0.0,0.0,1.0,1.0,3.0,9.0,1.0,1.0,4.0,3.0,0.0,2.0,43064,1364.2239,NaN,2790.0317,3714.9630,115,3.6431,43,1.3622,53,260,8.2365,52484,31787,1006.9799,NaN,51010,-0.068589,NaN,900.0,195,NaN,NaN,34.6885,16648.0,19255.0,365.0,15745.0,19908.0,359.0,903.0,-653.0,6.0,1.0,1.0,0.0,0.0,0.0,1.0,24376.0,29938.0,563.0,23707.0,31810.0,574.0,669.0,-1872.0,-11.0,1.0,1.0,0.0,0.0,0.0,1.0,1.750000,0.500000,1662.639916,-212.122492
67594,67594,ESPORTSTMNT03_3165687,complete,NaN,LCK,2023,Summer,0,2023-06-07 09:09:21,2,13.10,100,Blue,team,NaN,NaN,Hanwha Life Esports,oe:team:3a1d18f46bcb3716ebcfcf4ef068934,NaN,Yuumi,Maokai,Annie,Jax,Malphite,1771,1,21,8,46,21,8,5.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,0.7115,0.9825,1.0,4.0,0.0,4.0,0.0,1.0,0.0,2.0,0.0,1.0,0.0,NaN,0.0,0.0,1.0,2.0,0.0,1.0,2.0,0.0,1.0,11.0,2.0,1.0,1.0,5.0,3.0,4.0,0.0,64790,2195.0311,NaN,2398.4416,2116.2620,99,3.3540,58,1.9650,56,282,9.5539,64347,44905,1521.3439,NaN,56900,0.239937,NaN,863.0,209,NaN,NaN,36.3185,17573.0,19024.0,327.0,15633.0,18366.0,309.0,1940.0,658.0,18.0,5.0,7.0,3.0,3.0,2.0,5.0,26565.0,30325.0,510.0,24999.0,30125.0,514.0,1566.0,200.0,-4.0,7.0,8.0,5.0,5.0,3.0,7.0,8.375000,2.625000,2180.022586,495.652174
67595,67595,ESPORTSTMNT03_3165687,complete,NaN,LCK,2023,Summer,0,2023-06-07 09:09:21,2,13.10,200,Red,team,NaN,NaN,KT Rolster,oe:team:d723992568441c6bafa2658129eade7,NaN,Neeko,Zeri,Lucian,LeBlanc,Ahri,1771,0,8,21,12,8,21,1.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.2710,0.9825,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,2.0,11.0,0.0,0.0,3.0,5.0,0.0,4.0,42966,1455.6522,NaN,28

In [10]:
teams_data[
    ["gameid", "teamname", "gamelength", "totalgold", "gpm", "earned gpm", "gdpm"]
]

,gameid,teamname,gamelength,totalgold,gpm,earned gpm,gdpm
67582,ESPORTSTMNT03_3167508,KT Rolster,1894,59180,1874.762408,1219.1024,212.122492
67583,ESPORTSTMNT03_3167508,Hanwha Life Esports,1894,52484,1662.639916,1006.9799,-212.122492
67594,ESPORTSTMNT03_3165687,Hanwha Life Esports,1771,64347,2180.022586,1521.3439,495.652174
67595,ESPORTSTMNT03_3165687,KT Rolster,1771,49717,1684.370412,1025.6917,-495.652174
67630,ESPORTSTMNT03_3166614,KT Rolster,1605,50073,1871.887850,1208.3738,290.654206
...,...,...,...,...,...,...,...
74423,ESPORTSTMNT01_3380253,KT Rolster,1431,51585,2162.893082,1493.1237,616.226415
74470,ESPORTSTMNT01_3380258,Dplus KIA,1751,58427,2002.067390,1342.8555,272.724158
74471,ESPORTSTMNT01_3380258,Nongshim RedForce,1751,50468,1729.343232,1070.1314,-272.724158
74518,ESPORTSTMNT01_3381214,Nongshim RedForce,1878,51082,1632.012780,975.9744,-292.779553


In [11]:
cond = np.isinf(teams_data["kd"])
teams_data.loc[cond, "kd"] = teams_data.loc[cond, "kills"]
teams_data.loc[cond, "kd"]

67690    11.0
68338    11.0
69059    13.0
Name: kd, dtype: float64

In [12]:
teams_id = teams_data["teamid"].unique()
groupby = teams_data.groupby("teamid")
teams_stats = pd.DataFrame(index=teams_id)
teams_stats["Team"] = groupby.last()["teamname"]

columns = [
    "gamelength",
    "kills",
    "deaths",
    "assists",
    "kd",
    "firstblood",
    "team kpm",
    "ckpm",
    "firstdragon",
    "dragons",
    "opp_dragons",
    "elders",
    "opp_elders",
    "firstherald",
    "heralds",
    "opp_heralds",
    "firstbaron",
    "barons",
    "opp_barons",
    "firsttower",
    "towers",
    "opp_towers",
    "firstmidtower",
    "firsttothreetowers",
    "turretplates",
    "opp_turretplates",
    "inhibitors",
    "opp_inhibitors",
    "dpm",
    "damagetakenperminute",
    "wpm",
    "wcpm",
    "vspm",
    "gpm",
    "gdpm",
    "cspm",
    "golddiffat10",
    "xpdiffat10",
    "csdiffat10",
    "golddiffat15",
    "xpdiffat15",
    "csdiffat15",
]
teams_stats[columns] = groupby[columns].mean()
teams_stats["win_gamelength"] = (
    teams_data.loc[teams_data["result"] == 1].groupby("teamid")["gamelength"].mean()
)
teams_stats["loss_gamelength"] = (
    teams_data.loc[teams_data["result"] == 0].groupby("teamid")["gamelength"].mean()
)
teams_stats[['gamelength', 'win_gamelength', 'loss_gamelength']] /= 60


teams_stats.set_index("Team", inplace=True)
teams_stats

,gamelength,kills,deaths,assists,kd,firstblood,team kpm,ckpm,firstdragon,dragons,opp_dragons,elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,dpm,damagetakenperminute,wpm,wcpm,vspm,gpm,gdpm,cspm,golddiffat10,xpdiffat10,csdiffat10,golddiffat15,xpdiffat15,csdiffat15,win_gamelength,loss_gamelength
Team,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
KT Rolster,27.566667,11.700000,7.800000,28.400000,3.006429,0.600000,0.448450,0.721430,0.600000,2.400000,1.600000,0.000000,0.000000,0.500000,1.100000,0.900000,0.600000,0.600000,0.500000,0.600000,7.000000,4.200000,0.500000,0.600000,5.300000,4.700000,0.900000,0.600000,1975.524640,2585.012570,3.283290,1.257080,7.475850,1897.670635,139.590299,34.370640,376.100000,139.000000,5.300000,1175.100000,776.800000,9.900000,27.678571,27.305556
Hanwha Life Esports,31.258333,10.400000,7.200000,23.400000,3.270000,0.700000,0.347020,0.572810,0.500000,2.000000,2.300000,0.100000,0.000000,0.500000,0.900000,1.100000,0.500000,0.800000,0.500000,0.500000,6.100000,5.400000,0.400000,0.500000,3.800000,5.300000,1.200000,0.700000,2079.584220,2699.945560,3.382060,1.583660,8.616960,1800.922526,61.998795,34.000900,-116.800000,-201.500000,-5.800000,-425.800000,44.500000,-7.000000,33.161111,28.404167
Dplus KIA,31.118333,12.400000,6.800000,31.900000,3.221818,0.500000,0.407880,0.616470,0.800000,2.900000,1.500000,0.000000,0.100000,0.600000,1.100000,0.900000,0.500000,0.800000,0.400000,0.500000,6.800000,4.000000,0.600000,0.600000,5.700000,4.800000,1.300000,0.400000,2330.542290,2693.759840,3.339480,1.861560,8.834360,1879.879301,215.918230,33.921300,423.800000,498.400000,8.600000,1179.700000,808.500000,14.800000,30.052381,33.605556
Liiv SANDBOX,30.429167,8.375000,12.625000,21.875000,1.377936,0.375000,0.270962,0.695388,0.375000,1.375000,2.875000,0.000000,0.000000,0.500000,0.875000,1.125000,0.500000,0.500000,0.750000,0.250000,4.125000,7.500000,0.375000,0.250000,3.625000,5.625000,0.625000,1.250000,1808.503375,2856.017000,3.794313,1.781250,8.805375,1707.778170,-175.019549,33.051500,-670.500000,-253.500000,1.250000,-1689.625000,-1440.125000,-5.750000,27.675000,31.347222
Kwangdong Freecs,33.885417,10.000000,9.500000,25.000000,1.528299,0.750000,0.283062,0.562775,0.750000,3.375000,1.375000,0.250000,0.125000,0.750000,1.250000,0.750000,0.125000,0.250000,1.125000,0.875000,6.625000,5.000000,1.000000,0.875000,6.000000,3.000000,0.875000,0.625000,2123.401588,3079.897825,3.484875,1.731400,8.790662,1800.505682,53.768135,34.535250,950.625000,599.375000,19.000000,2073.625000,719.000000,23.625000,33.479167,34.291667
DRX,29.514815,6.555556,16.666667,15.222222,0.400089,0.111111,0.197456,0.793767,0.000000,0.666667,3.444444,0.111111,0.111111,0.222222,0.777778,1.222222,0.333333,0.555556,0.666667,0.222222,2.333333,8.000000,0.222222,0.222222,3.888889,6.444444,0.111111,1.444444,1946.215667,3301.084400,3.353044,1.474889,7.611133,1572.103559,-406.418388,29.964922,-1152.555556,-583.333333,-27.333333,-2867.777778,-1192.555556,-43.555556,34.433333,28.900000
OKSavingsBank BRION,31.323333,5.500000,8.900000,12.400000,1.770865,0.600000,0.174150,0.471470,0.400000,1.700000,2.700000,0.000000,0.100000,0.400000,0.900000,1.100000,0.500000,0.500000,0.700000,0.300000,3.700000,6.700000,0.400000,0.400000,4.000000,5.000000,0.400000,1.000000,1713.528790,2702.431850,3.321150,1.773730,8.319640,1674.528986,-181.922727,33.839480,-123.000000,-333.700000,-11.800000,-678.500000,-378.100000,-10.400000,32.822222,30.680952
Nongshim RedForce,29.977778,6.111111,10.555556,13.000000,0.940717,0.333333,0.202511,0.559800,0.444444,1.555556,2.444444,0.000000,0.000000,0.555556,1.000000,1.000000,0.111111,0.222222,1.000000,0.333333,3.000000,8.222222,0.111111,0.111111,2.888889,5.000000,0.333333,1.555556,1604.290033,2980.389256,3.779133,1.582689,8.201533,1669.705131,-226.318810,33.161144,-532.555556,-524.888889,-7.000000,-1206.555556,-1551.888889,-26.555556,

In [13]:
origin_columns = {
    "combat": ["kd", "firstblood", "team kpm", "dpm", "damagetakenperminute"],
    "objects": [
        "firstdragon",
        "dragons",
        "firstherald",
        "heralds",
        "firstbaron",
        "barons",
        "elders",
    ],
    "towers": [
        "firsttower",
        "towers",
        "firstmidtower",
        "firsttothreetowers",
        "turretplates",
    ],
    "macro": [
        "gamelength",
        "win_gamelength",
        "loss_gamelength",
        "gpm",
        "gdpm",
        "wpm",
        "wcpm",
        "vspm",
    ],
    "early": [
        "golddiffat10",
        "xpdiffat10",
        "csdiffat10",
        "golddiffat15",
        "xpdiffat15",
        "csdiffat15",
    ],
}
columns = {}

for key, values in origin_columns.items():
    columns[key] = ["_" + col for col in values]
    scaler = MinMaxScaler()
    teams_stats[columns[key]] = scaler.fit_transform(teams_stats[values])
teams_stats.sort_index(inplace=True)
teams_stats.head()

,gamelength,kills,deaths,assists,kd,firstblood,team kpm,ckpm,firstdragon,dragons,opp_dragons,elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,turretplates,opp_turretplates,inhibitors,opp_inhibitors,dpm,damagetakenperminute,wpm,wcpm,vspm,gpm,gdpm,cspm,golddiffat10,xpdiffat10,csdiffat10,golddiffat15,xpdiffat15,csdiffat15,win_gamelength,loss_gamelength,_kd,_firstblood,_team kpm,_dpm,_damagetakenperminute,_firstdragon,_dragons,_firstherald,_heralds,_firstbaron,_barons,_elders,_firsttower,_towers,_firstmidtower,_firsttothreetowers,_turretplates,_gamelength,_win_gamelength,_loss_gamelength,_gpm,_gdpm,_wpm,_wcpm,_vspm,_golddiffat10,_xpdiffat10,_csdiffat10,_golddiffat15,_xpdiffat15,_csdiffat15
Team,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DRX,29.514815,6.555556,16.666667,15.222222,0.400089,0.111111,0.197456,0.793767,0.0,0.666667,3.444444,0.111111,0.111111,0.222222,0.777778,1.222222,0.333333,0.555556,0.666667,0.222222,2.333333,8.0,0.222222,0.222222,3.888889,6.444444,0.111111,1.444444,1946.215667,3301.08440,3.353044,1.474889,7.611133,1572.103559,-406.418388,29.964922,-1152.555556,-583.333333,-27.333333,-2867.777778,-1192.555556,-43.555556,34.433333,28.900000,0.000000,0.000000,0.084964,0.470808,1.000000,0.000,0.000000,0.000000,0.111111,0.377358,0.491803,0.444444,0.000000,0.000000,0.1250,0.145455,0.226700,0.308312,1.000000,0.255441,0.000000,0.000000,0.392119,0.360324,0.099582,0.000000,0.000000,0.000000,0.000000,0.126424,0.000000
Dplus KIA,31.118333,12.400000,6.800000,31.900000,3.221818,0.500000,0.407880,0.616470,0.8,2.900000,1.500000,0.000000,0.100000,0.600000,1.100000,0.900000,0.500000,0.800000,0.400000,0.500000,6.800000,4.0,0.600000,0.600000,5.700000,4.800000,1.300000,0.400000,2330.542290,2693.75984,3.339480,1.861560,8.834360,1879.879301,215.918230,33.921300,423.800000,498.400000,8.600000,1179.700000,808.500000,14.800000,30.052381,33.605556,0.756061,0.608696,0.852096,1.000000,0.151866,1.000,0.824615,0.715789,0.571429,0.660377,0.852459,0.000000,0.425532,0.788235,0.5500,0.640000,0.637280,0.562084,0.351770,0.902253,0.900658,0.982303,0.373433,1.000000,1.000000,0.749510,0.914624,0.775540,0.819095,0.830454,0.868638
Gen.G,28.193333,11.500000,8.300000,27.400000,4.132232,0.700000,0.407580,0.698790,0.5,2.400000,1.600000,0.000000,0.000000,0.400000,0.700000,1.300000,0.700000,0.900000,0.200000,0.600000,7.000000,3.7,0.500000,0.600000,5.200000,5.200000,1.100000,0.400000,2026.293360,2738.80486,3.068400,1.603030,8.181840,1913.826839,203.749706,34.902230,144.800000,262.600000,5.900000,143.800000,504.800000,18.400000,28.481250,27.041667,1.000000,0.921739,0.851003,0.581070,0.214772,0.625,0.640000,0.336842,0.000000,1.000000,1.000000,0.000000,0.578723,0.823529,0.4375,0.640000,0.523929,0.099176,0.119297,0.000000,1.000000,0.963096,0.000000,0.572310,0.519680,0.616854,0.715251,0.717266,0.609458,0.723603,0.922225
Hanwha Life Esports,31.258333,10.400000,7.200000,23.400000,3.270000,0.700000,0.347020,0.572810,0.5,2.000000,2.300000,0.100000,0.000000,0.500000,0.900000,1.100000,0.500000,0.800000,0.500000,0.500000,6.100000,5.4,0.400000,0.500000,3.800000,5.300000,1.200000,0.700000,2079.584220,2699.94556,3.382060,1.583660,8.616960,1800.922526,61.998795,34.000900,-116.800000,-201.500000,-5.800000,-425.800000,44.500000,-7.000000,33.161111,28.404167,0.768971,0.921739,0.630222,0.654448,0.160505,0.625,0.492308,0.526316,0.285714,0.660377,0.852459,0.400000,0.425532,0.664706,0.3250,0.509091,0.206549,0.584240,0.811755,0.187285,0.669603,0.739355,0.432091,0.540266,0.839972,0.492471,0.322847,0.464748,0.494187,0.561656,0.544139
KT Rolster,27.566667,11.700000,7.800000,28.400000,3.006429,0.600000,0.448450,0.721430,0.6,2.400000,1.600000,0.000000,0.000000,0.500000,1.100000,0.900000,0.600000,0.600000,0.500000,0.600000,7.000000,4.2,0.500000,0.600000,5.300000,4.700000,0.900000,0.600000,1975.524640,2585.01257,3.283290,1.257080,7.475850,1897.670635,139.590299,34.370640,

In [14]:
ROWS, COLUMNS = get_rows_columns(teams_stats, len(origin_columns))
min_value, max_value = 0, 1

fig = get_fig_subplots(teams_stats, ROWS, COLUMNS)
for i, team_name in enumerate(teams_stats.index):
    for j, (key, cols) in enumerate(columns.items()):
        df = teams_stats.loc[team_name, cols]
        theta_list = get_theta_list(teams_stats, team_name, origin_columns[key])

        fig.add_trace(
            go.Scatterpolar(r=df, theta=theta_list, fill="toself", name=team_name),
            row=i + 1,
            col=j + 1,
        )

fig.update_layout(
    height=400 * ROWS,
    width=550 * COLUMNS,
    showlegend=False,
    title=dict(y=1),
)
fig.update_annotations(yshift=20)
fig.update_polars(
    radialaxis=dict(range=[min_value - 0.2, max_value + 0.2], visible=False)
)
fig.show()

In [ ]:
players_data = matches.loc[~matches["playername"].isna()]
players_data.head()

In [ ]:
cond = np.isinf(players_data["kda"])
players_data.loc[cond, "kda"] = players_data.loc[cond, ["kills", "assists"]].sum(axis=1)
players_data.loc[cond, "kda"]

In [ ]:
groupby = players_data.groupby(["playerid", "position"])
idx = groupby.groups.keys()
idx = pd.MultiIndex.from_tuples(idx, names=["playerid", "position"])
players_stats = pd.DataFrame(index=idx)
players_stats[["Player", "Team"]] = groupby.last()[["playername", "teamname"]]

mean_columns = [
    "kills",
    "deaths",
    "assists",
    "dpm",
    "damageshare",
    "damagetakenperminute",
    "wpm",
    "wcpm",
    "vspm",
    "gpm",
    "cspm",
    "golddiffat10",
    "xpdiffat10",
    "csdiffat10",
    "golddiffat15",
    "xpdiffat15",
    "csdiffat15",
    "kda",
]

players_stats[mean_columns] = groupby[mean_columns].mean()
players_stats